# Few-shot Learning with Multilingual Language Models (XGLM)

## Introduction

In this work, we train a family of multilingual generative language models, dubbed XGLM, on a balanced corpus covering a diverse set of languages, and study their few- and zero-shot learning capabilities in a wide range of tasks. Our largest model with 7.5 billion parameters sets new state of the art in few-shot learning on more than 20 representative languages, outperforming GPT-3 of comparable size in multilingual commonsense reasoning (+7.4 accuracy points for 0-shot, +9.4 for 4-shot) and natural language inference (+5.4 for 0-shot, +5.4 for 4-shot). We have included a [model card](model_card.md) of XGLM for transparency and accountability.



## Data and Languages
XGLM models are trained on a new multilingual corpus extracted from CommonCrawl (CC100-XL), a significantly larger multilingual dataset covering 68 Common Crawl (CC) snapshots (from [Summer 2013](http://commoncrawl.org/2013/11/new-crawl-data-available/) to [March/April 2020](https://commoncrawl.org/2020/04/march-april-2020-crawl-archive-now-available/) consisting of 134 languages. The detailed languages and data statistics are reported in the paper (Table A.1).



## Pre-trained models

Model | Layers | Model Dim | FFN Dim | Languages | Download
---|---|---|---|---|---
`XGLM 564M` | 24 | 1024 | 4096 | trained on 30 languages|  [xglm.564M.tar.gz](https://dl.fbaipublicfiles.com/fairseq/models/xglm/xglm.564M.tar.gz)
`XGLM 1.7B` | 24 | 2048 | 8192 | trained on 30 languages|  [xglm.1.7B.tar.gz](https://dl.fbaipublicfiles.com/fairseq/models/xglm/xglm.1.7B.tar.gz)
`XGLM 2.9B` | 48 | 2048 | 8192 | trained on 30 languages|  [xglm.2.9B.tar.gz](https://dl.fbaipublicfiles.com/fairseq/models/xglm/xglm.2.9B.tar.gz)
`XGLM 7.5B` | 32 | 4096 | 16384 | trained on 30 languages|  [xglm.7.5B.tar.gz](https://dl.fbaipublicfiles.com/fairseq/models/xglm/xglm.7.5B.tar.gz)
`XGLM 4.5B` | 48 | 2048 | 16384 | trained on 134 languages|  [xglm.4.5B.tar.gz](https://dl.fbaipublicfiles.com/fairseq/models/xglm/xglm.4.5B.tar.gz)

## Evaluation



In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from transformers import XGLMTokenizer, XGLMForCausalLM

tokenizer = XGLMTokenizer.from_pretrained("facebook/xglm-564M")
model = XGLMForCausalLM.from_pretrained("facebook/xglm-564M")
model.eval()
model.cuda()

XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): Embedding(256008, 1024, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
      (1): XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_pr

### XCOPA

In [ ]:
from datasets import load_dataset

langs_xcopa = ["et", "ht", "it", "id", "qu", "sw", "zh", "ta", "th", "tr", "vi"]

xcopa = {}
for lang in langs_xcopa:
    xcopa[lang] = load_dataset("xcopa", lang)

In [ ]:
xcopa["et"]["validation"][0]

{'premise': 'Mees keeras kraani lahti.',
 'choice1': 'Tualett täitus veega.',
 'choice2': 'Tilast voolas vett.',
 'question': 'effect',
 'label': 1,
 'idx': 0,
 'changed': False}

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import pandas as pd

def get_logprobs(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    input_ids, output_ids = inputs["input_ids"], inputs["input_ids"][:, 1:]
    outputs = model(**inputs, labels=input_ids)
    logits = outputs.logits
    logprobs = torch.gather(F.log_softmax(logits, dim=2), 2, output_ids.unsqueeze(2))
    return logprobs

# Zero-shot evaluation for the Choice of Plausible Alternatives (COPA) task.
# A return value of 0 indicates that the first alternative is more plausible,
# while 1 indicates that the second alternative is more plausible.
def XCOPA_eval(prompt, alternative1, alternative2):
    lprob1 = get_logprobs(prompt + "\n" + alternative1).sum()
    lprob2 = get_logprobs(prompt + "\n" + alternative2).sum()
    return 0 if lprob1 > lprob2 else 1

results_xcopa = {"idx": xcopa["et"]["test"]["idx"], 
           "label": xcopa["et"]["test"]["label"]}
for lang in langs_xcopa:
    predictions = []
    for idx, example in tqdm(enumerate(xcopa[lang]["test"])):
        predict = XCOPA_eval(example["premise"], example["choice1"], example["choice2"])
        predictions.append(predict)
    results_xcopa[lang] = predictions

500it [00:54,  9.15it/s]
500it [00:50,  9.83it/s]
500it [00:50,  9.84it/s]
500it [00:50,  9.95it/s]
500it [00:51,  9.74it/s]
500it [00:50,  9.82it/s]
500it [00:50,  9.82it/s]
500it [00:55,  9.01it/s]
500it [00:50,  9.89it/s]
500it [00:50,  9.84it/s]
500it [00:50,  9.84it/s]


In [ ]:
results_xcopa_df = pd.DataFrame(results_xcopa).to_csv("XCOPA_xglm-564M.tsv", sep="\t", index=False)

In [ ]:
results_xcopa_df = pd.read_csv("XCOPA_xglm-564M.tsv", delimiter="\t")

In [ ]:
results_xcopa_df

,idx,label,et,ht,it,id,qu,sw,zh,ta,th,tr,vi
0,0,0,1,1,1,1,1,1,1,1,1,1,1
1,1,0,1,1,1,1,1,1,1,1,1,1,1
2,2,1,0,0,0,1,0,0,0,0,1,0,0
3,3,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,1,0,1,0,0,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,1,1,1,1,1,1,0,0,0,1,1,1
496,496,1,0,0,0,0,0,0,1,0,0,0,1
497,497,0,0,1,0,0,0,1,1,0,1,1,0
498,498,1,0,1,1,1,1,0,1,1,1,0,1


In [ ]:
accuracy = {}
for lang in langs_xcopa:
    compare = results_xcopa_df["label"] == results_xcopa_df[lang]
    acc = list(compare).count(True) / len(list(compare)) * 100
    accuracy[lang] = round(acc, 2)

accuracy

{'et': 52.4,
 'ht': 54.2,
 'it': 52.0,
 'id': 55.8,
 'qu': 49.2,
 'sw': 52.8,
 'zh': 53.2,
 'ta': 54.4,
 'th': 55.6,
 'tr': 53.0,
 'vi': 55.8}

### XStoryCloze

In [3]:
from datasets import load_dataset

langs_xstory = ["en", "ru", "zh", "es", "ar", "hi", "id", "te", "sw", "eu", "my"]

x_story_cloze = {}
for lang in langs_xstory:
    x_story_cloze[lang] = load_dataset('juletxara/xstory_cloze', lang)

Reusing dataset xstory_cloze (/gaueko0/users/jetxaniz007/.cache/huggingface/datasets/juletxara___xstory_cloze/en/0.0.0/aa2197eb1953f6f2eb1df88a38ec6595b7e13bc9f2af795c7db065b5b0959ba3)


  0%|          | 0/2 [00:00<?, ?it/s]

Reusing dataset xstory_cloze (/gaueko0/users/jetxaniz007/.cache/huggingface/datasets/juletxara___xstory_cloze/ru/0.0.0/aa2197eb1953f6f2eb1df88a38ec6595b7e13bc9f2af795c7db065b5b0959ba3)


  0%|          | 0/2 [00:00<?, ?it/s]

Reusing dataset xstory_cloze (/gaueko0/users/jetxaniz007/.cache/huggingface/datasets/juletxara___xstory_cloze/zh/0.0.0/aa2197eb1953f6f2eb1df88a38ec6595b7e13bc9f2af795c7db065b5b0959ba3)


...

  0%|          | 0/2 [00:00<?, ?it/s]

Reusing dataset xstory_cloze (/gaueko0/users/jetxaniz007/.cache/huggingface/datasets/juletxara___xstory_cloze/eu/0.0.0/aa2197eb1953f6f2eb1df88a38ec6595b7e13bc9f2af795c7db065b5b0959ba3)


  0%|          | 0/2 [00:00<?, ?it/s]

Reusing dataset xstory_cloze (/gaueko0/users/jetxaniz007/.cache/huggingface/datasets/juletxara___xstory_cloze/my/0.0.0/aa2197eb1953f6f2eb1df88a38ec6595b7e13bc9f2af795c7db065b5b0959ba3)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
x_story_cloze["en"]["train"][0]

{'story_id': '138d5bfb-05cc-41e3-bf2c-fa85ebad14e2',
 'input_sentence_1': 'Rick grew up in a troubled household.',
 'input_sentence_2': 'He never found good support in family, and turned to gangs.',
 'input_sentence_3': "It wasn't long before Rick got shot in a robbery.",
 'input_sentence_4': 'The incident caused him to turn a new leaf.',
 'sentence_quiz1': 'He is happy now.',
 'sentence_quiz2': 'He joined a gang.',
 'answer_right_ending': 1}

In [5]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import pandas as pd

def get_logprobs(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    input_ids, output_ids = inputs["input_ids"], inputs["input_ids"][:, 1:]
    outputs = model(**inputs, labels=input_ids)
    logits = outputs.logits
    logprobs = torch.gather(F.log_softmax(logits, dim=2), 2, output_ids.unsqueeze(2))
    return logprobs

def XStoryCloze_eval(prompt, alternative1, alternative2):
    lprob1 = get_logprobs(prompt + " " + alternative1).mean()
    lprob2 = get_logprobs(prompt + " " + alternative2).mean()
    ppl1 = torch.exp(-lprob1)
    ppl2 = torch.exp(-lprob2)
    pred = 1 if lprob1 > lprob2 else 2
    return pred, lprob1, lprob2, ppl1, ppl2

In [7]:
size = len(x_story_cloze["en"]["eval"])
results_xstory = {"idx": list(range(size)), 
           "label": x_story_cloze["en"]["eval"]["answer_right_ending"]}
for lang in langs_xstory:
    predictions, lprobs1, lprobs2, ppls1, ppls2 = [], [], [], [], []
    for idx, example in tqdm(enumerate(x_story_cloze[lang]["eval"]), total=size):
        input_sentences = example["input_sentence_1"] + " " + example["input_sentence_2"] + " " + example["input_sentence_3"] + " " + example["input_sentence_4"]
        pred, lprob1, lprob2, ppl1, ppl2 = XStoryCloze_eval(input_sentences, example["sentence_quiz1"], example["sentence_quiz2"])
        predictions.append(pred)
        lprobs1.append(round(lprob1.item(), 2))
        lprobs2.append(round(lprob2.item(), 2))
        ppls1.append(round(ppl1.item(), 2))
        ppls2.append(round(ppl2.item(), 2))
    results_xstory[lang] = predictions
    results_xstory[lang + "_lprob1"] = lprobs1
    results_xstory[lang + "_lprob2"] = lprobs2
    results_xstory[lang + "_ppl1"] = ppls1
    results_xstory[lang + "_ppl2"] = ppls2

1511it [02:05, 12.01it/s]
1511it [02:03, 12.21it/s]
1511it [02:11, 11.53it/s]
1511it [02:17, 10.97it/s]
1511it [02:21, 10.66it/s]
1511it [02:24, 10.44it/s]
1511it [02:14, 11.24it/s]
1511it [02:21, 10.67it/s]
1511it [02:18, 10.92it/s]
1511it [02:15, 11.15it/s]
1511it [02:18, 10.94it/s]


In [9]:
results_xstory_df = pd.DataFrame(results_xstory).to_csv("../results/XStoryCloze_xglm-564M.tsv", sep="\t", index=False)

In [10]:
results_xstory_df = pd.read_csv("../results/XStoryCloze_xglm-564M.tsv", delimiter="\t")

In [25]:
accuracy = {}
for lang in langs_xstory:
    compare = results_xstory_df["label"] == results_xstory_df[lang]
    acc = list(compare).count(True) / len(list(compare)) * 100
    accuracy[lang] = round(acc, 1)
accuracy["avg"] = round(sum(accuracy.values()) / len(accuracy), 1)

In [31]:
perplexity_correct = {}
perplexity_incorrect = {}

for lang in langs_xstory:
    correct = []
    incorrect = []
    for idx, row in results_xstory_df.iterrows():
        if row["label"] == 1:
            correct.append(row[lang + "_ppl1"])
            incorrect.append(row[lang + "_ppl2"])
        else:
            correct.append(row[lang + "_ppl2"])
            incorrect.append(row[lang + "_ppl1"])
    perplexity_correct[lang] = round(sum(correct) / len(correct), 2)
    perplexity_incorrect[lang] = round(sum(incorrect) / len(incorrect), 2)
perplexity_correct["avg"] = round(sum(perplexity_correct.values()) / len(perplexity_correct), 2)
perplexity_incorrect["avg"] = round(sum(perplexity_incorrect.values()) / len(perplexity_incorrect), 2)

In [32]:
metrics_df = pd.DataFrame({"acc": accuracy, "ppl_cor": perplexity_correct, "ppl_inc": perplexity_incorrect})
metrics_df

,acc,ppl_cor,ppl_inc
en,61.3,22.79,23.98
ru,59.4,22.81,23.64
zh,56.2,35.40,36.31
es,56.5,26.19,27.03
ar,53.9,48.08,48.80
hi,54.1,24.63,25.00
id,56.9,46.55,48.30
te,56.9,30.85,31.76
sw,56.4,51.94,53.58
eu,55.1,37.03,38.09


In [33]:
metrics_df.to_csv("../results/XStoryCloze_xglm-564M_metrics.tsv", sep="\t", index_label="lang")